In [10]:
%reset
# All imports
from pyparsing import Word, hexnums, WordEnd, Optional, alphas, alphanums
from collections import defaultdict
import pandas as pd
from copy import copy
import csv
import pandas as pd 
import numpy as np
import csv
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.metrics import accuracy_score

# Grid search cross-validation for tuning hyperparameters
from sklearn.model_selection import GridSearchCV

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [11]:
address_format = Word(hexnums, exact=8) + WordEnd() # use WordEnd to avoid parsing leading a-f of non-hex numbers as a hex
byte_format = Word(hexnums, exact=2) + WordEnd()
instrn_line_format = ".text:" + address_format + (byte_format*(1,))("bytes") + Word(alphas,alphanums)("instruction")
byte_line_format = address_format + (byte_format*(1,))("bytes")

In [12]:
# Globals
SAMPLES_BASE_DIR = 'trainingData/train/'
TRAIN_FILES = ['EZ5uGxW3X8DvKMTcJ6Id','j7UqgpTCiRr1s6kWHvKB','EdwQ1my8754NBkMaJxGA','jRtAmhHUTa3Gud5E07vf','EhCda1DLfb93XTwc8uYJ','jUeRolnr49Mm7DF0JIZX','EiD3lRHWhCw4zY9SjXVI','jZfoAgI4BUzQne1lY5hP','EpBVNQnoMzYaCbe96OZx','jZzi1wtyQf5L7HuTO9hU','EsWOrxC0pMt9jgPQb7cA']
# VALIDATE_FILES = []
INSTRN_BIGRAM_THRESHOLD = 20
BYTE_BIGRAM_THRESHOLD = 100

In [13]:
def get_features(filename):
    instrn_unigram = defaultdict(int)
    instrn_bigram = defaultdict(int)
    byte_unigram = defaultdict(int)
    byte_bigram = defaultdict(int)
    segments = defaultdict(int)
    with open(SAMPLES_BASE_DIR + filename + ".asm", 'r', encoding='Latin-1') as file:
        prev, now = 0, 0
        for line in file:
            # Filtering lines
            segments[line.split(':')[0]] += 1
            if not line.startswith('.text'):
                continue
            if ' db ' in line or ' dd ' in line or ' dw ' in line or 'align ' in line:
                continue
                
            try:
                result = instrn_line_format.parseString(line)
            except:
                continue
                
            prev = now
            now = result.instruction
            instrn_bigram[(prev, now)] += 1
            instrn_unigram[now] += 1
#                 if result.instruction == 'CC':
#                     print(line)
    instrn_bigram = defaultdict(int, {k:v for k,v in instrn_bigram.items() if v > INSTRN_BIGRAM_THRESHOLD and k[0] != 0})
#     print(segments)
#     print(instrn_unigram)
#     print(sum(instrn_unigram.values()))
#     print("==========================================================================================")
#     print(instrn_bigram)
#     print("==========================================================================================")
    with open(SAMPLES_BASE_DIR + filename + ".bytes", 'r', encoding='Latin-1') as file:
        prev, now = 0, 0
        for line in file:
            try:
                result = byte_line_format.parseString(line)
            except:
                continue
            
            byte_list = list(result.bytes)
            for byte in byte_list:
                prev = now
                now = byte
                byte_bigram[(prev, now)] += 1
                byte_unigram[now] += 1

    byte_bigram = defaultdict(int, {k:v for k,v in byte_bigram.items() if v > BYTE_BIGRAM_THRESHOLD and k[0] != 0})
#     print(byte_unigram)
#     print(sum(byte_unigram.values()))
#     print("==========================================================================================")
#     print(byte_bigram)
#     print("==========================================================================================")
    all_features = copy(segments)
    all_features.update(instrn_unigram)
    all_features.update(instrn_bigram)
    all_features.update(byte_unigram)
    all_features.update(byte_bigram)
    p = pd.DataFrame(all_features, index=[filename,])
#     print(p)
    print(filename)
    return p 

In [14]:
def get_labels():
    label = defaultdict(int)
    with open("./trainLabels.csv", 'r') as file:
        fileReader = csv.reader(file, delimiter=',')
        fileReader = list(fileReader)
        fileReader = fileReader[1:]
        for row in fileReader:
            label[row[0]] = row[1]
        trainLabels = {k:v for k,v in label.items() if k in TRAIN_FILES}
        trainLabels = pd.DataFrame(list(trainLabels.values()), index = trainLabels.keys(), columns = ['Label'])
        print(trainLabels)
    return trainLabels

In [15]:
def get_train_data():
    train_data_points_ = pd.DataFrame()
    for filename in TRAIN_FILES:
        features = get_features(filename)
        train_data_points_ = pd.concat([train_data_points_, features], axis=0)
    train_data_points_.fillna(0, inplace=True)
    train_data_labels_ = get_labels()
    return (train_data_points_,train_data_labels_)

In [16]:
def scaler(train_data_):
    scaler = MinMaxScaler()
    scaled_train_data_ = scaler.fit_transform(train_data_)
    return scaled_train_data_

In [28]:
def linearClassifier(train_data_points,train_data_labels):
    c = 1
    penalty = 'l2'
    solver = 'newton-cg'
    multi_class = 'ovr'
    clf = linear_model.LogisticRegression(penalty=penalty,C=c,solver=solver,multi_class=multi_class)
#     gsclf = GridSearchCV(clf, {}, cv=2, scoring='accuracy')
#     gsclf.fit(train_data_points,train_data_labels)
    clf.fit(train_data_points,train_data_labels)
#     return gsclf
    return clf

In [18]:
train_data_points_,train_data_labels_ = get_train_data()
# validate_data_points_,validate_data_labels_ = get_test_data()
train_data_points_ = scaler(train_data_points_)
model = linearClassifier(train_data_points_,train_data_labels_)
print(model.best_score_)
# accuracy = checkAccuracy(model, validate_data_points_, validate_data_labels)
# print(accuracy)

EZ5uGxW3X8DvKMTcJ6Id
j7UqgpTCiRr1s6kWHvKB
EdwQ1my8754NBkMaJxGA
jRtAmhHUTa3Gud5E07vf
EhCda1DLfb93XTwc8uYJ
jUeRolnr49Mm7DF0JIZX
EiD3lRHWhCw4zY9SjXVI
jZfoAgI4BUzQne1lY5hP
EpBVNQnoMzYaCbe96OZx
jZzi1wtyQf5L7HuTO9hU
EsWOrxC0pMt9jgPQb7cA
                     Label
EsWOrxC0pMt9jgPQb7cA     8
jZfoAgI4BUzQne1lY5hP     9
jUeRolnr49Mm7DF0JIZX     2
EpBVNQnoMzYaCbe96OZx     4
EZ5uGxW3X8DvKMTcJ6Id     1
EdwQ1my8754NBkMaJxGA     1
jZzi1wtyQf5L7HuTO9hU     3
EhCda1DLfb93XTwc8uYJ     1
EiD3lRHWhCw4zY9SjXVI     8
j7UqgpTCiRr1s6kWHvKB     3
jRtAmhHUTa3Gud5E07vf     8


NameError: name 'train_data_points' is not defined

In [27]:
train_data_points_ = scaler(train_data_points_)
model = linearClassifier(train_data_points_,train_data_labels_)
# print(model.best_score_)
print(train_data_points_)


[[0.23045478 0.56809339 0.19373777 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.04696673 ... 0.         0.         0.        ]
 ...
 [0.         0.19455253 0.         ... 0.         0.         0.        ]
 [0.         0.         0.04403131 ... 0.         0.         0.        ]
 [0.17986714 0.         0.02054795 ... 1.         1.         1.        ]]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/u